In [ ]:
from cobra import Reaction
import escher
import escher.urls
import cobra
import cobra.test
import json
import os
import string
from IPython.display import HTML
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis.parsimonious import optimize_minimal_flux
model = cobra.io.load_matlab_model('/Users/LAURENCE/Desktop/Senior Design/RECON1', 'RECON1')

## 6/24/16 changes


Can you try setting the lower bound on 
FTHFLi, 
FTHFL, and 
FTHFLm to 0 and recalculating the flux?

The ones to set the lower bound to 0 are FTHFLi and FHTHLmi.

# 6/16/16 changes
#Omar re-ran MATLAB calculate fluxes and got new values. Objective set as ATP isntead of growth
#DM_ATP = 7.9939, biomass = 0.0177, EX_glc = -0.4882 --> result: not a huge change. need to look into constraining AA

EX_arg__L_e = 0.0022
EX_asn__L_e = -0.0058
EX_asp__L_e = -0.0081
EX_chol_e = -6.2386e-04
EX_gln__L_e = -0.1202
EX_gly_e =  -0.0019
EX_ile__L_e = -0.0081
EX_leu__L_e =   -0.0110
EX_lys__L_e =   -0.0111
EX_phe__L_e =  -0.0040
EX_pro__L_e =  0.0045
EX_ser__L_e =  -0.0167
EX_trp__L_e = -0.0013
EX_tyr__L_e = -0.0068
EX_val__L_e =  -0.0079


In [ ]:
# Changes:
# nh4 is now also -1000, and changed glucose to -100
# Set "MMMm" and "OCOAT1m" rxn bounds to 0
#http://localhost:8888/notebooks/Desktop/Senior%20Design/FH_model_troubleshoot_systematic_v03.ipynb# Also, make sure EX_pi_e is also -1000, not sure if it is or not, but it shouldn't be limiting.
# 5/26/16 shut off SUCOASm when setting up model
# 5/26/16 Remove 5aop sink to verify
# 5/28/16 turned SUCOA1sm back on
# 6/4/16 Added DM_atp_c demand and constrained the lb to 0.9*1.07 or 0.0177 as obtained from maxing DM_atp_c via calculatefluxes
# 6/8/16 tried to fix objective -- going to max DM_atp_c
# 6/11/16 constrained glucose to -0.4293 and growth to 0.018

In [ ]:
#Changes round 2:
# (2.1) 6/14/16 -- make BILDGLCURte and BILGLCURte irreversible forward by setting lb to 0 since it's infeasible
# (2.2)6/14/16 Replace all h_c's with h_i's in the reactions ATPS4m, CYOOm2, NADH2-u10m, CYOR-u10m

Dan -- 06/03

First, use the metabolite uptakes and growth rates from one of the NCI60 lines (back from calculate fluxes). I'm not sure that it will matter which one you use.
Be sure to constrain the growth rate both lower and upper bounds to the same value (with the 'b' argument).
Then, change the objective to DM_atp_c and maximize that - extract a value of maybe like 0.9 of that maximum and constrain the lower bound on DM_atp_c to that value.
Finally, change the objective back to growth and do pFBA to estimate the flux state. See if the TCA cycle looks more 'normal' now.
If things are looking more normal, retry the ranking analysis and see if that had an effect (probably still want to KO both FUM and FUMm).

### 5/28 Dan's directions: we need to fix the TCA cycle of the normal model.
#### Action: Figure out where PDH flux is going -- what fluxes are using accoa_m

* ACACT1m
* ACCOACrm
* ACGSm
* ASPNATm
* GLYATm
* HMGCOASim

### More systematic way of troubleshooting the FH model

#### Overview of FH case:
KO of mitochondrial FH should lead to build up of succoa. This buildup of succoa should feed into the heme synthesis cycle and cause an increase in heme synthesis pathway.

#### Problem with our model: 
We see no increase in pheme

#### Troubleshooting efforts thus far:
* Set up the model to try to turn on the heme pathway.
* KO both FUM and FUMm since the model may be trying to use both.
* Track the flow of metabolites starting from malate, through succoa, through the heme synthesis pathway

#### New troubleshooting trial and errors:
* (1) Going to try KO FUM (non-mitochondrial) in healthy state since there is 0 flux going through FUMm -- a modeling eror since TCA cycle is in the mitochondria. Result: We see a noticeable backup in the TCA cycle from FUMm to SUCD1m. However, there is no flux going through SUCOAS1m. Action: Need to find source of where succ is leaving and force it through SUCOA1sm

* (2) Going to find where else succ_m can go since it's not going into SUCOAS1m. Results: All flux from SUCD1m is going into SUCCtm. Action: Shut off SUCCtm when setting up the model and observe

* (3) Shut off SUCCtm and run model. Results: Flux is properly forced through heme synthesis when FH is KO.

#### Results after finishing (3):
Pheme is properly synthesized after FH KO. However, no pheme synthesis in normal healthy state so there is no "differential" but only an activation of pheme production.


In [ ]:
#creat ethanalamine demand
#metabolite is etha_c

DM_etha_c = Reaction('DM_etha_c')
DM_etha_c.name = DM_etha_c

etha_c = model.metabolites.get_by_id('etha_c')

DM_etha_c.add_metabolites({etha_c: -1})
model.add_reaction(DM_etha_c)

#create a transporter for this

Tyr_ggnt = Reaction('Tyr_ggnt')
Tyr_ggnt.name = Tyr_ggnt

Tyr_ggn_e = model.metabolites.get_by_id('Tyr_ggn_e')
Tyr_ggn_c = model.metabolites.get_by_id('Tyr_ggn_c')

Tyr_ggnt.add_metabolites({Tyr_ggn_e: -1, Tyr_ggn_c: 1})
model.add_reaction(Tyr_ggnt)

DM_etha_c.lower_bound = -1
Tyr_ggnt.lower_bound = -1000

except_EX_names = '''DM_etha_c, EX_peplys_e, EX_Tyr_ggn_e, EX_arg__L_e, EX_asn__L_e, EX_asp__L_e, EX_chol_e, EX_cl_e, EX_glc__D_e, EX_gln__L_e, EX_gly_e, EX_h_e, EX_h2o_e, EX_ile__L_e, EX_k_e, EX_leu__L_e, EX_lys__L_e, EX_na1_e, EX_nh4_e, EX_o2_e, EX_phe__L_e, EX_pi_e, EX_pro__L_e, EX_ser__L_e, EX_thr__L_e, EX_trp__L_e, EX_tyr__L_e, EX_val__L_e'''
except_EX_names_split = except_EX_names.split(', ')
print except_EX_names_split

# Changing all lower bounds to 0 except the given reactions above

for a in model.reactions[1188:1591]:
    a.lower_bound = 0

model.reactions.get_by_id('EX_yvite_e').lower_bound = 0
model.reactions.get_by_id('EX_10fthf5glu_e').lower_bound = 0
    
for b in model.reactions[1188:1591]:
    for c in range(len(except_EX_names_split)):  
        if b.id  == except_EX_names_split[c]:
            b.lower_bound = -1
            
biomass_NCI60 = Reaction('biomass_NCI60')
biomass_NCI60.name = biomass_NCI60

ala_L_c = model.metabolites.get_by_id('ala__L_c')
arg_L_c = model.metabolites.get_by_id('arg__L_c')
asn_L_c = model.metabolites.get_by_id('asn__L_c')
asp_L_c = model.metabolites.get_by_id('asp__L_c')
atp_c = model.metabolites.get_by_id('atp_c')
clpn_hs_c = model.metabolites.get_by_id('clpn_hs_c')
ctp_c = model.metabolites.get_by_id('ctp_c')
dag_hs_c = model.metabolites.get_by_id('dag_hs_c')
datp_c = model.metabolites.get_by_id('datp_c')
dctp_c = model.metabolites.get_by_id('dctp_c')
dgtp_c = model.metabolites.get_by_id('dgtp_c')
dttp_c = model.metabolites.get_by_id('dttp_c')
gln_L_c = model.metabolites.get_by_id('gln__L_c')
glu_L_c = model.metabolites.get_by_id('glu__L_c')
gly_c = model.metabolites.get_by_id('gly_c')
glygn2_c = model.metabolites.get_by_id('glygn2_c')
gtp_c = model.metabolites.get_by_id('gtp_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
hdca_c = model.metabolites.get_by_id('hdca_c')
hdcea_c = model.metabolites.get_by_id('hdcea_c')
ile_L_c = model.metabolites.get_by_id('ile__L_c')
leu_L_c = model.metabolites.get_by_id('leu__L_c')
lpchol_hs_c = model.metabolites.get_by_id('lpchol_hs_c')
lys_L_c = model.metabolites.get_by_id('lys__L_c')
mag_hs_c = model.metabolites.get_by_id('mag_hs_c')
ocdca_c = model.metabolites.get_by_id('ocdca_c')
ocdcea_c = model.metabolites.get_by_id('ocdcea_c')
pa_hs_c = model.metabolites.get_by_id('pa_hs_c')
pail_hs_c = model.metabolites.get_by_id('pail_hs_c')
pchol_hs_c = model.metabolites.get_by_id('pchol_hs_c')
pe_hs_c = model.metabolites.get_by_id('pe_hs_c')
phe_L_c = model.metabolites.get_by_id('phe__L_c')
ps_hs_c = model.metabolites.get_by_id('ps_hs_c')
ser_L_c = model.metabolites.get_by_id('ser__L_c')
sphmyln_hs_c = model.metabolites.get_by_id('sphmyln_hs_c')
thr_L_c = model.metabolites.get_by_id('thr__L_c')
trp_L_c = model.metabolites.get_by_id('trp__L_c')
tyr_L_c = model.metabolites.get_by_id('tyr__L_c')
utp_c = model.metabolites.get_by_id('utp_c')
val_L_c = model.metabolites.get_by_id('val__L_c')
pro_L_m = model.metabolites.get_by_id('pro__L_m')
chsterol_r = model.metabolites.get_by_id('chsterol_r')
xolest_hs_r = model.metabolites.get_by_id('xolest_hs_r')
adp_c = model.metabolites.get_by_id('adp_c')
h_c = model.metabolites.get_by_id('h_c')
pi_c = model.metabolites.get_by_id('pi_c')

#remove ocdca and ocdcea
# added xolest value to cholesterol in biomass and remove xolest

biomass_NCI60.add_metabolites({ala_L_c: -0.587929, arg_L_c: -0.380280, asn_L_c: -0.323313, asp_L_c: -0.261396, 
                               atp_c: -35.033540, clpn_hs_c: -0.000624, ctp_c: -0.033435, dag_hs_c: -0.001032, 
                               datp_c: -0.014557, dctp_c: -0.009770, dgtp_c: -0.009748, dttp_c: -0.014546,
                               gln_L_c: -0.319051, glu_L_c: -0.387401, gly_c: -0.504294, glygn2_c: -0.034479,
                               gtp_c: -0.055967, h2o_c: -35.000000, hdca_c: -0.008293, hdcea_c: -0.003315, 
                               ile_L_c: -0.319813, leu_L_c: -0.548692, lpchol_hs_c: -0.002470, lys_L_c: -0.552717, 
                               mag_hs_c: -0.001456, pa_hs_c: -0.010645,
                               pail_hs_c: -0.005016, pchol_hs_c: -0.022878, pe_hs_c: -0.018211, phe_L_c: -0.170743,
                               ps_hs_c: -0.006808, ser_L_c: -0.385852, sphmyln_hs_c: -0.010215, thr_L_c: -0.378004, 
                               trp_L_c: -0.039847, tyr_L_c: -0.150141, utp_c: -0.063323, val_L_c: -0.385554,
                               pro_L_m: -0.237850, chsterol_r: -0.054102,  adp_c: 35.000000, 
                               h_c: 35.000000, pi_c: 35.000000})

model.add_reaction(biomass_NCI60)
my_objective = model.reactions.get_by_id('biomass_NCI60')
model.change_objective(my_objective)

### Adding and constraining DM_atp_c


atp+h2o->adp+pi+h


In [ ]:
# Adding DM_atp_c reaction

DM_atp_c = Reaction('DM_atp_c')
DM_atp_c.name = DM_atp_c

atp_c = model.metabolites.get_by_id('atp_c')
adp_c = model.metabolites.get_by_id('adp_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
h_c = model.metabolites.get_by_id('h_c')
pi_c = model.metabolites.get_by_id('pi_c')


DM_atp_c.add_metabolites({atp_c: -1, h2o_c: -1, adp_c: 1, pi_c: 1, h_c: 1})
model.add_reaction(DM_atp_c)

# Setting lower bounds for DM_atp_c as per Dan

#model.reactions.get_by_id('DM_atp_c').lower_bound = 0.9*7.9939

### Feeding the cell w fe2, o2, h, and nh4 for heme

In [ ]:
model.reactions.get_by_id('EX_h_e').lower_bound = -1000
model.reactions.get_by_id('EX_o2_e').lower_bound = -1000
model.reactions.get_by_id('EX_fe2_e').lower_bound = -1000
model.reactions.get_by_id('EX_nh4_e').lower_bound = -1000

### Setting glucose to -100 (to -0.4293  as per calculate fluxes 6/11/11)

In [ ]:
model.reactions.get_by_id('EX_glc__D_e').lower_bound = -0.4882

In [ ]:
# constrained as per calculate fluxes 
model.reactions.get_by_id('biomass_NCI60').lower_bound = 0.0177
model.reactions.get_by_id('biomass_NCI60').upper_bound = 0.0177

### Setting EX_pi_e to not be limiting, initially lb was -1

In [ ]:
model.reactions.get_by_id('EX_pi_e').lower_bound = -1000

In [ ]:
# 6/16/16 Changing amino acid uptake constraints

model.reactions.get_by_id('EX_arg__L_e').lower_bound = -0.0022

model.reactions.get_by_id('EX_asn__L_e').lower_bound = -0.0058
model.reactions.get_by_id('EX_asp__L_e').lower_bound = -0.0081
model.reactions.get_by_id('EX_chol_e').lower_bound = -0.00062386
model.reactions.get_by_id('EX_gln__L_e').lower_bound = -0.1202
model.reactions.get_by_id('EX_gly_e').lower_bound = -0.0019
model.reactions.get_by_id('EX_ile__L_e').lower_bound = -0.0081
model.reactions.get_by_id('EX_leu__L_e').lower_bound = -0.0110
model.reactions.get_by_id('EX_lys__L_e').lower_bound = -0.0111
model.reactions.get_by_id('EX_phe__L_e').lower_bound = -0.0040

model.reactions.get_by_id('EX_pro__L_e').lower_bound = -0.0045

model.reactions.get_by_id('EX_ser__L_e').lower_bound = -0.0167
model.reactions.get_by_id('EX_trp__L_e').lower_bound = -0.0013
model.reactions.get_by_id('EX_tyr__L_e').lower_bound = -0.0068
model.reactions.get_by_id('EX_val__L_e').lower_bound = -0.0079

In [ ]:
#6/24/16 lowerbound changes

model.reactions.get_by_id('FTHFLi').lower_bound = 0
model.reactions.get_by_id('FTHFLmi').lower_bound = 0


## (1)  KO FUM non mitochondrial

In [ ]:
model.reactions.get_by_id('FUM').lower_bound = 0
model.reactions.get_by_id('FUM').upper_bound = 0

# 6/14/16 Changes

## (2.1) Make BILGLCURte and the D one irreversible forward by setting lb to 0 (6/14/16)

In [ ]:
model.reactions.get_by_id('BILGLCURte').lower_bound = 0
model.reactions.get_by_id('BILDGLCURte').lower_bound = 0
print model.reactions.get_by_id('BILGLCURte').lower_bound

## (2.2) Trying to add metabolite h_i to model

In [ ]:
h_i = Metabolite('h_i', formula='h', name='h_i', compartment='i')
model.add_metabolites(h_i)

## (2.2) Add CYOOm2 and replace h_c's with h_i's

#### ATPS4m

In [ ]:
print "initial:   " + model.reactions.get_by_id('ATPS4m').reaction
# delete reaction
model.remove_reactions('ATPS4m')

# re-add the reaction while replacing h_c with h_i
ATPS4m = Reaction('ATPS4m')
ATPS4m.name = ATPS4m

adp_m = model.metabolites.get_by_id('adp_m')
h_i = model.metabolites.get_by_id('h_i')
pi_m = model.metabolites.get_by_id('pi_m')
h2o_m = model.metabolites.get_by_id('h2o_m')
h_m = model.metabolites.get_by_id('h_m')
atp_m = model.metabolites.get_by_id('atp_m')

ATPS4m.add_metabolites({adp_m: -1, h_i: -4, pi_m: -1, h2o_m: 1, h_m: 3, atp_m: 1})
model.add_reaction(ATPS4m)

print "replaced with:   " + model.reactions.get_by_id('ATPS4m').reaction

#### Deleting CYOOm3

In [ ]:
model.remove_reactions('CYOOm3')

#### Adding CYOOm2

In [ ]:
CYOOm2 = Reaction('CYOOm2')
CYOOm2.name = CYOOm2

h_m = model.metabolites.get_by_id('h_m')
o2_m = model.metabolites.get_by_id('o2_m')
focytC_m = model.metabolites.get_by_id('focytC_m')
h_i = model.metabolites.get_by_id('h_i')
h2o_m = model.metabolites.get_by_id('h2o_m')
ficytC_m = model.metabolites.get_by_id('ficytC_m')


CYOOm2.add_metabolites({h_m: -8, o2_m: -1, focytC_m: -4, h_i: 4, h2o_m: 2, ficytC_m: 4})
model.add_reaction(CYOOm2)

print "rxn:   " + model.reactions.get_by_id('CYOOm2').reaction

#### Replacing NADH2_u10m

In [ ]:
print "initial:   " + model.reactions.get_by_id('NADH2_u10m').reaction
# delete reaction
model.remove_reactions('NADH2_u10m')

# re-add the reaction while replacing h_c with h_i
NADH2_u10m = Reaction('NADH2_u10m')
NADH2_u10m.name = NADH2_u10m

q10_m = model.metabolites.get_by_id('q10_m')
h_m = model.metabolites.get_by_id('h_m')
nadh_m = model.metabolites.get_by_id('nadh_m')
nad_m = model.metabolites.get_by_id('nad_m')
h_i = model.metabolites.get_by_id('h_i')
q10h2_m = model.metabolites.get_by_id('q10h2_m')



NADH2_u10m.add_metabolites({q10_m: -1, h_m: -5, nadh_m: -1, nad_m: 1, h_i: 4, q10h2_m: 1})
model.add_reaction(NADH2_u10m)

print "replaced with:   " + model.reactions.get_by_id('NADH2_u10m').reaction

#### Replacing CYOR_u10m

In [ ]:
print "initial:   " + model.reactions.get_by_id('CYOR_u10m').reaction
# delete reaction
model.remove_reactions('CYOR_u10m')

# re-add the reaction while replacing h_c with h_i
CYOR_u10m = Reaction('CYOR_u10m')
CYOR_u10m.name = NADH2_u10m

ficytC_m = model.metabolites.get_by_id('ficytC_m')
h_m = model.metabolites.get_by_id('h_m')
q10h2_m = model.metabolites.get_by_id('q10h2_m')
q10_m = model.metabolites.get_by_id('q10_m')
focytC_m = model.metabolites.get_by_id('focytC_m')
h_i = model.metabolites.get_by_id('h_i')


CYOR_u10m.add_metabolites({ficytC_m: -2, h_m: -2, q10h2_m: -1, q10_m: 1, focytC_m: 2, h_i: 4})
model.add_reaction(CYOR_u10m)

print "replaced with:   " + model.reactions.get_by_id('CYOR_u10m').reaction

In [ ]:
p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

In [ ]:
print "the normal flux of FH is %f in a healthy cell when optimized for growth" %(p_solution.x_dict['FUMm'])

### Adding Malate (mal__L_m) sink reaction that has the same flux as normal FH flux determined via pFBA

In [ ]:
'''mal__L_m = model.metabolites.get_by_id('mal__L_m')
mal__L_m_sink = Reaction('mal__L_m_sink')
mal__L_m_sink.name = mal__L_m_sink
mal__L_m_sink.add_metabolites({mal__L_m: -1})
model.add_reaction(mal__L_m_sink)'''

In [ ]:
'''# constraining the sink flux to that of normal FH flux
model.reactions.get_by_id("mal__L_m_sink").upper_bound = p_solution.x_dict['FUM']
model.reactions.get_by_id("mal__L_m_sink").lower_bound = p_solution.x_dict['FUM']'''

### Constraining FH flux

In [ ]:
model.reactions.get_by_id("FUMm").upper_bound = p_solution.x_dict['FUMm']
model.reactions.get_by_id("FUMm").lower_bound = p_solution.x_dict['FUMm']

In [ ]:
print model.reactions.get_by_id("FUMm").upper_bound
print model.reactions.get_by_id("FUMm").lower_bound

## Create and set  5aop_m  flux thing as objective

In [ ]:
'''fiveaop_m = model.metabolites.get_by_id('5aop_m')
fiveaop_m_sink = Reaction('fiveaop_m_sink')
fiveaop_m_sink.name = fiveaop_m_sink
fiveaop_m_sink.add_metabolites({fiveaop_m: -1})
model.add_reaction(fiveaop_m_sink)

my_objective = model.reactions.get_by_id('fiveaop_m_sink')
model.change_objective(my_objective)'''

## pFBA with 5aop and malate sink

In [ ]:
'''p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)'''

## Knocking out FH

In [ ]:
# FH KO
model.reactions.get_by_id("FUM").upper_bound = 0
model.reactions.get_by_id("FUM").lower_bound = 0

model.reactions.get_by_id("FUMm").upper_bound = 0
model.reactions.get_by_id("FUMm").lower_bound = 0

In [ ]:
# Now calculate the pFBA
p_solution_FHKO = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

In [ ]:
# This is the soln producing pheme
print p_solution.x_dict['FCLTm']
print p_solution_FHKO.x_dict['FCLTm']

## Algorithim init flux

In [ ]:
# working copy!
# This creates an array of modified met names since I'm getting errors accessing model.metabolites[i].id in some of the code


newmet_array = [0] * len(model.metabolites)
met_count = 0
for i in range(len(model.metabolites)):
    newmet_array[met_count] = string.replace(model.metabolites[i].id, " ", "_")
    met_count = met_count + 1

In [ ]:
# creates an empty dict to store DM_rxns as keys and the p_solution.f values
normal_maxmet_dict = {}

for i in range(len(model.metabolites)):

    # this part creates a demand reaction for each metabolite
    temp_DM = 'DM_' + str(newmet_array[i])
    
    # this nested for loop tries to catch if a demand reaction already exists in the model
    for ii in model.reactions:
        if temp_DM == str(ii):
            # this part sets the newly created demand as the objective
            my_objective = model.reactions.get_by_id(str(temp_DM))
            model.change_objective(my_objective)

            # this part runs pFBA on the new demand
            p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

            # this stores the demand flux into a dictionary where its demand rxn is the key
            normal_maxmet_dict[str(temp_DM)] = p_solution.f

            # this deletes the demand at the end of each iteration
            model.reactions.get_by_id(str(temp_DM)).delete()
            
    temp_DM = Reaction(str(temp_DM))
    newmet_array[i] = model.metabolites.get_by_id(model.metabolites[i].id)
    temp_DM.add_metabolites({newmet_array[i]: -1})
    model.add_reaction(temp_DM)
    
    # this part sets the newly created demand as the objective
    my_objective = model.reactions.get_by_id(str(temp_DM))
    model.change_objective(my_objective)
    
    # this part runs pFBA on the new demand
    p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)
    
    # this stores the demand flux into a dictionary where its demand rxn is the key
    normal_maxmet_dict[str(temp_DM)] = p_solution.f
    
    # this deletes the demand at the end of each iteration
    model.reactions.get_by_id(str(temp_DM)).delete()

## Algorithm KO flux

In [ ]:
# FUMm KO

model.reactions.get_by_id("FUMm").upper_bound = 0
model.reactions.get_by_id("FUMm").lower_bound = 0

In [ ]:
# working copy!
# This creates an array of modified met names since I'm getting errors accessing model.metabolites[i].id in some of the code


newmet_array = [0] * len(model.metabolites)
met_count = 0
for i in range(len(model.metabolites)):
    newmet_array[met_count] = string.replace(model.metabolites[i].id, " ", "_")
    met_count = met_count + 1

In [ ]:
# creates an empty dict to store DM_rxns as keys and the p_solution.f values
ARGSSKO_maxmet_dict = {}

for i in range(len(model.metabolites)):
    
    # this part creates a demand reaction for each metabolite
    temp_DM = 'DM_' + str(newmet_array[i])
    
    # this nested for loop tries to catch if a demand reaction already exists in the model
    for ii in model.reactions:
        if temp_DM == str(ii):
            # this part sets the newly created demand as the objective
            my_objective = model.reactions.get_by_id(str(temp_DM))
            model.change_objective(my_objective)

            # this part runs pFBA on the new demand
            p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

            # this stores the demand flux into a dictionary where its demand rxn is the key
            ARGSSKO_maxmet_dict[str(temp_DM)] = p_solution.f

            # this deletes the demand at the end of each iteration
            model.reactions.get_by_id(str(temp_DM)).delete()
            
    temp_DM = Reaction(str(temp_DM))
    newmet_array[i] = model.metabolites.get_by_id(model.metabolites[i].id)
    temp_DM.add_metabolites({newmet_array[i]: -1})
    model.add_reaction(temp_DM)
    
    # this part sets the newly created demand as the objective
    my_objective = model.reactions.get_by_id(str(temp_DM))
    model.change_objective(my_objective)
    
    # this part runs pFBA on the new demand
    p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)
    
    # this stores the demand flux into a dictionary where its demand rxn is the key
    ARGSSKO_maxmet_dict[str(temp_DM)] = p_solution.f
    
    # this deletes the demand at the end of each iteration
    model.reactions.get_by_id(str(temp_DM)).delete()

In [ ]:
# This combines the normal_flux solutions and KO_flux solutions into one dictionary
# The same key, the reaction, and stores 2 flux values as a list to the dic key where [0] is normal and [1] is KO

combined_dic = {key: {'init_flux': normal_maxmet_dict[key], 'KO_flux': ARGSSKO_maxmet_dict[key]} for key in ARGSSKO_maxmet_dict if key in normal_maxmet_dict}

In [ ]:
G1 = {}
G2 = {}
G3 = {}

for rxn, flux in combined_dic.iteritems():
    # if both init and final are 0, stored into group 1
    if flux['init_flux'] == 0 and flux['KO_flux'] == 0:
        G1[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}
        
    # if both init and final are non-zero, stored into group 1
    if flux['init_flux'] != 0 and flux['KO_flux'] != 0:
        G1[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}
        
    # if init is 0 and final is non-zero, stored into group 2
    if flux['init_flux'] == 0 and flux['KO_flux'] != 0:
        G2[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}
        
    # if init is non-zero and final is 0, stored into group 3
    if flux['init_flux'] != 0 and flux['KO_flux'] == 0:
        G3[rxn] = {'init_flux': flux['init_flux'], 'KO_flux': flux['KO_flux']}

In [ ]:
# Execute divisions using this function
# This catches divisions of denominator 0 and just outputs 0
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

In [ ]:
# This is the first step of normalization
# This is the flux of KO - initial flux
G1_output_norm_dic = {a: {'init_flux': b['init_flux'], 'KO_flux': b['KO_flux'], 'normalized_flux':
                       safe_div((b['KO_flux']-b['init_flux']),b['init_flux']), 
                          'absolute_normalized_flux': abs(safe_div((b['KO_flux']-b['init_flux']),b['init_flux']))} for a,b in G1.iteritems()}

In [ ]:
# Ranks based on criteria by creating an ordered dictionary
# G1 -- normalized flux ranking (ratio)
import operator
import collections


G1_descending = collections.OrderedDict(sorted(G1_output_norm_dic.items(),
                                               key=lambda kv: kv[1]['absolute_normalized_flux'], reverse=True))


In [ ]:
# writing ARGSSKO_maxmet_dict as a .csv file

metabolite_demand_dic = [0] * len(ARGSSKO_maxmet_dict)
init_flux_out_dic = [0] * len(ARGSSKO_maxmet_dict)
KO_flux_out_dic = [0] * len(ARGSSKO_maxmet_dict)
normalized_flux_out_dic = [0] * len(ARGSSKO_maxmet_dict)
absolute_normalized_flux_out_dic = [0] * len(ARGSSKO_maxmet_dict)

metabolite_demand_count = 0
init_flux_count = 0
KO_flux_count = 0
normalized_flux_count = 0
absolute_normalized_flux_count = 0

for key, value in G1_descending.iteritems():
    metabolite_demand_dic[metabolite_demand_count] = key
    metabolite_demand_count = metabolite_demand_count + 1
    
for a, b in G1_descending.iteritems():
    init_flux_out_dic[init_flux_count] = b['init_flux']
    init_flux_count = init_flux_count + 1
    
for c, d in G1_descending.iteritems():
    KO_flux_out_dic[KO_flux_count] = d['KO_flux']
    KO_flux_count = KO_flux_count + 1
    

for e, f in G1_descending.iteritems():
    normalized_flux_out_dic[normalized_flux_count] = f['normalized_flux']
    normalized_flux_count = normalized_flux_count + 1  
    
for g, h in G1_descending.iteritems():
    absolute_normalized_flux_out_dic[absolute_normalized_flux_count] = h['absolute_normalized_flux']
    absolute_normalized_flux_count = absolute_normalized_flux_count + 1  


In [ ]:
# this saves the array into a .csv file as named

import csv
from itertools import izip

with open('FH_G1_ranked_fluxes.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['metabolite_demand', 'init_flux', 'KO_flux', 'normalized_flux', 'absolute_normalized_flux'])
    writer.writerows(izip(metabolite_demand_dic, init_flux_out_dic, KO_flux_out_dic, normalized_flux_out_dic, absolute_normalized_flux_out_dic))

In [ ]:
p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

In [ ]:
b = escher.Builder(map_json='/Users/LAURENCE/Desktop/Senior Design/RECON1.Central.json',
                   reaction_data=p_solution.x_dict,
                   # color and size according to the absolute value
                   reaction_styles=['color', 'size', 'abs', 'text'],
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': 'red', 'size': 4},
                                   {'type': 'mean', 'color': 'green', 'size': 20},
                                   {'type': 'max', 'color': 'blue', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook(scroll_behavior='pan')